In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [2]:
df_preprocessed = pd.read_csv('../data/processed/preprocessed.csv.gz', compression='gzip')
df_preprocessed.shape

C:\Users\tharaka_50498.DIALOG\AppData\Local\Temp\1\ipykernel_2940\545262313.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_preprocessed = pd.read_csv('../data/processed/preprocessed.csv.gz', compression='gzip')


(1526659, 125)

Post process steps
* handle outliers
* handling categorical columns (onehot encode or suitable method)
* scaling numerical features
* encoding categorical variables

In [3]:
df_preprocessed.drop(columns='bankacctype_710L', inplace=True)
df_preprocessed['annuitynextmonth_57A'].fillna(0,inplace=True)
df_preprocessed['currdebt_22A'].fillna(0,inplace=True)
df_preprocessed['currdebtcredtyperange_828A'].fillna(0,inplace=True)
df_preprocessed.drop(columns='maxdbddpdtollast6m_4187119P', inplace=True)

df_preprocessed.drop(columns='date_decision', inplace=True)
df_preprocessed['credtype_322L'] = df_preprocessed['credtype_322L'].fillna(df_preprocessed['credtype_322L'].mode()[0])
df_preprocessed['disbursementtype_67L'] = df_preprocessed['disbursementtype_67L'].fillna(df_preprocessed['disbursementtype_67L'].mode()[0])

C:\Users\tharaka_50498.DIALOG\AppData\Local\Temp\1\ipykernel_2940\1113466415.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_preprocessed['annuitynextmonth_57A'].fillna(0,inplace=True)
C:\Users\tharaka_50498.DIALOG\AppData\Local\Temp\1\ipykernel_2940\1113466415.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are set

In [4]:
columns_for_freq_encode = ['lastst_736L','lastapprcommoditycat_1041M','lastapprcommoditytypec_5251766M','lastcancelreason_561M','lastrejectcommoditycat_161M','lastrejectcommodtypec_5251769M','lastrejectreason_759M','lastrejectreasonclient_4145040M','previouscontdistrict_112M']
columns_for_one_hot_encode = ['cardtype_51L','inittransactioncode_186L','credtype_322L','disbursementtype_67L']

In [5]:
len(columns_for_one_hot_encode) + len(columns_for_freq_encode)

13

In [6]:
df_preprocessed.shape

(1526659, 122)

In [7]:
df_preprocessed.head()

,case_id,MONTH,WEEK_NUM,target,datelastunpaid_3546854D,cardtype_51L,cntincpaycont9m_3716944L,cntpmts24_3658933L,commnoinclast6m_3546845L,eir_270L,...,numinstregularpaid_973L.1,numinsttopaygr_769L.1,numinstunpaidmax_3546851L.1,pmtnum_254L.1,posfpd30lastmonth_3976960P.1,price_1097A.1,sumoutstandtotal_3546847A.1,totaldebt_9A.1,totalsettled_863A.1,time_delta
0,0,201901,0,0,2019-01-03,NOCARD,0.0,0.0,0.0,0.4500,...,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,15384.0
1,1,201901,0,0,2019-01-03,NOCARD,0.0,0.0,0.0,0.2999,...,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,15384.0
2,2,201901,0,0,2019-01-04,NOCARD,0.0,0.0,0.0,0.4500,...,0.0,0.0,0.0,36.0,0.0,0.0,0.0,0.0,0.0,15360.0
3,3,201901,0,0,2019-01-03,NOCARD,0.0,0.0,0.0,0.4200,...,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,15384.0
4,4,201901,0,1,2019-01-04,NOCARD,0.0,0.0,0.0,0.4500,...,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,15360.0


In [8]:
# separate categorical and numerical cols
numerical_cols = df_preprocessed.select_dtypes(include='number').columns.tolist()
print(len(numerical_cols))
categorical_cols = df_preprocessed.select_dtypes(include='object').columns.tolist()
print(len(categorical_cols))

106
14


In [9]:
numerical_cols

['case_id',
 'MONTH',
 'WEEK_NUM',
 'target',
 'cntincpaycont9m_3716944L',
 'cntpmts24_3658933L',
 'commnoinclast6m_3546845L',
 'eir_270L',
 'interestrate_311L',
 'lastapprcredamount_781A',
 'lastrejectcredamount_222A',
 'maininc_215A',
 'maxannuity_159A',
 'numinstls_657L',
 'numinstlsallpaid_934L',
 'numinstpaidearly_338L',
 'numinstregularpaid_973L',
 'numinsttopaygr_769L',
 'numinstunpaidmax_3546851L',
 'pmtnum_254L',
 'posfpd30lastmonth_3976960P',
 'price_1097A',
 'sumoutstandtotal_3546847A',
 'totaldebt_9A',
 'totalsettled_863A',
 'actualdpdtolerance_344P',
 'amtinstpaidbefduel24m_4187115A',
 'annuity_780A',
 'annuitynextmonth_57A',
 'applicationcnt_361L',
 'applications30d_658L',
 'applicationscnt_1086L',
 'applicationscnt_464L',
 'applicationscnt_629L',
 'applicationscnt_867L',
 'avgdbddpdlast24m_3658932P',
 'avgdbddpdlast3m_4187120P',
 'avgdbdtollast24m_4525197P',
 'avgdpdtolclosure24_3658938P',
 'avginstallast24m_3658937A',
 'avgmaxdpdlast9m_3716943P',
 'avgoutstandbalancel6m

In [10]:
cols_to_remove = ['case_id', 'MONTH','WEEK_NUM','target']

for value in cols_to_remove:
    numerical_cols.remove(value)

In [11]:
numerical_cols

['cntincpaycont9m_3716944L',
 'cntpmts24_3658933L',
 'commnoinclast6m_3546845L',
 'eir_270L',
 'interestrate_311L',
 'lastapprcredamount_781A',
 'lastrejectcredamount_222A',
 'maininc_215A',
 'maxannuity_159A',
 'numinstls_657L',
 'numinstlsallpaid_934L',
 'numinstpaidearly_338L',
 'numinstregularpaid_973L',
 'numinsttopaygr_769L',
 'numinstunpaidmax_3546851L',
 'pmtnum_254L',
 'posfpd30lastmonth_3976960P',
 'price_1097A',
 'sumoutstandtotal_3546847A',
 'totaldebt_9A',
 'totalsettled_863A',
 'actualdpdtolerance_344P',
 'amtinstpaidbefduel24m_4187115A',
 'annuity_780A',
 'annuitynextmonth_57A',
 'applicationcnt_361L',
 'applications30d_658L',
 'applicationscnt_1086L',
 'applicationscnt_464L',
 'applicationscnt_629L',
 'applicationscnt_867L',
 'avgdbddpdlast24m_3658932P',
 'avgdbddpdlast3m_4187120P',
 'avgdbdtollast24m_4525197P',
 'avgdpdtolclosure24_3658938P',
 'avginstallast24m_3658937A',
 'avgmaxdpdlast9m_3716943P',
 'avgoutstandbalancel6m_4187114A',
 'avgpmtlast12m_4525200A',
 'clien

Outlier Handeling

In [12]:
# use winsorization to handle outliers 
# it clips extream values to specified percentile
def winsorize_outliers(df, column, lower_pct=0.05, upper_pct=0.95):
    lower_bound = df[column].quantile(lower_pct)
    upper_bound = df[column].quantile(upper_pct)
    df[column] = df[column].clip(lower=lower_bound, upper=upper_bound)
    return df

# def winsorize_outsr(df, column, lower_pct=0.05, upper_pct=0.95):
#     lower_bound = daqw
#     df[co2lumn] = df[column].clip(lower=lower_bound, upper=upper_bound)
#     return df


# winsorize num cols
for col in numerical_cols:
    df_preprocessed = winsorize_outliers(df_preprocessed, col)

In [13]:
df_preprocessed.shape

(1526659, 122)

In [14]:
df_preprocessed.head()

,case_id,MONTH,WEEK_NUM,target,datelastunpaid_3546854D,cardtype_51L,cntincpaycont9m_3716944L,cntpmts24_3658933L,commnoinclast6m_3546845L,eir_270L,...,numinstregularpaid_973L.1,numinsttopaygr_769L.1,numinstunpaidmax_3546851L.1,pmtnum_254L.1,posfpd30lastmonth_3976960P.1,price_1097A.1,sumoutstandtotal_3546847A.1,totaldebt_9A.1,totalsettled_863A.1,time_delta
0,0,201901,0,0,2019-01-03,NOCARD,0.0,0.0,0.0,0.4500,...,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,15384.0
1,1,201901,0,0,2019-01-03,NOCARD,0.0,0.0,0.0,0.2999,...,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,15384.0
2,2,201901,0,0,2019-01-04,NOCARD,0.0,0.0,0.0,0.4500,...,0.0,0.0,0.0,36.0,0.0,0.0,0.0,0.0,0.0,15360.0
3,3,201901,0,0,2019-01-03,NOCARD,0.0,0.0,0.0,0.4200,...,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,15384.0
4,4,201901,0,1,2019-01-04,NOCARD,0.0,0.0,0.0,0.4500,...,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,15360.0


In [15]:
# fn freq encoding
def frequency_encode(column):
    frequency_map = column.value_counts(normalize=True)
    return column.map(frequency_map)

In [16]:
# one-hot 
one_hot_encoded = pd.get_dummies(df_preprocessed[columns_for_one_hot_encode])

# freq encode
for col in columns_for_freq_encode:
    df_preprocessed[col] = frequency_encode(df_preprocessed[col])

# sclae num
scaler = StandardScaler()
scaled_numerical = scaler.fit_transform(df_preprocessed[numerical_cols])
scaled_numerical_df = pd.DataFrame(scaled_numerical, columns=[f'scaled_{col}' for col in numerical_cols])

In [17]:
# combine all
df_postprocessed = pd.concat([df_preprocessed[['case_id','MONTH','WEEK_NUM','target']], one_hot_encoded, df_preprocessed[columns_for_freq_encode], scaled_numerical_df], axis=1)


In [18]:
df_postprocessed.shape

(1526659, 127)

In [19]:
df_postprocessed.head()

,case_id,MONTH,WEEK_NUM,target,cardtype_51L_INSTANT,cardtype_51L_NOCARD,cardtype_51L_PERSONALIZED,inittransactioncode_186L_CASH,inittransactioncode_186L_NDF,inittransactioncode_186L_POS,...,scaled_numinstregularpaid_973L.1,scaled_numinsttopaygr_769L.1,scaled_numinstunpaidmax_3546851L.1,scaled_pmtnum_254L.1,scaled_posfpd30lastmonth_3976960P.1,scaled_price_1097A.1,scaled_sumoutstandtotal_3546847A.1,scaled_totaldebt_9A.1,scaled_totalsettled_863A.1,scaled_time_delta
0,0,201901,0,0,False,True,False,True,False,False,...,-0.804146,-0.525393,-0.535457,0.869697,0.0,-1.037614,-0.488194,-0.48857,-0.756542,-0.069199
1,1,201901,0,0,False,True,False,True,False,False,...,-0.804146,-0.525393,-0.535457,0.184843,0.0,-1.037614,-0.488194,-0.48857,-0.756542,-0.069199
2,2,201901,0,0,False,True,False,True,False,False,...,-0.804146,-0.525393,-0.535457,2.239406,0.0,-1.037614,-0.488194,-0.48857,-0.756542,-0.070655
3,3,201901,0,0,False,True,False,True,False,False,...,-0.804146,-0.525393,-0.535457,-0.500012,0.0,-1.037614,-0.488194,-0.48857,-0.756542,-0.069199
4,4,201901,0,1,False,True,False,True,False,False,...,-0.804146,-0.525393,-0.535457,0.869697,0.0,-1.037614,-0.488194,-0.48857,-0.756542,-0.070655


In [20]:
# df_postprocessed.to_csv('data/processed/postprocessed_before_outlier_handled.csv.gz', compression='gzip')

In [22]:
df_postprocessed.to_csv('../data/processed/postprocessed_after_outlier_handled.csv.gz', compression='gzip', index=False)

In [23]:
df_postprocessed.shape

(1526659, 127)

In [24]:
df_postprocessed.head()

,case_id,MONTH,WEEK_NUM,target,cardtype_51L_INSTANT,cardtype_51L_NOCARD,cardtype_51L_PERSONALIZED,inittransactioncode_186L_CASH,inittransactioncode_186L_NDF,inittransactioncode_186L_POS,...,scaled_numinstregularpaid_973L.1,scaled_numinsttopaygr_769L.1,scaled_numinstunpaidmax_3546851L.1,scaled_pmtnum_254L.1,scaled_posfpd30lastmonth_3976960P.1,scaled_price_1097A.1,scaled_sumoutstandtotal_3546847A.1,scaled_totaldebt_9A.1,scaled_totalsettled_863A.1,scaled_time_delta
0,0,201901,0,0,False,True,False,True,False,False,...,-0.804146,-0.525393,-0.535457,0.869697,0.0,-1.037614,-0.488194,-0.48857,-0.756542,-0.069199
1,1,201901,0,0,False,True,False,True,False,False,...,-0.804146,-0.525393,-0.535457,0.184843,0.0,-1.037614,-0.488194,-0.48857,-0.756542,-0.069199
2,2,201901,0,0,False,True,False,True,False,False,...,-0.804146,-0.525393,-0.535457,2.239406,0.0,-1.037614,-0.488194,-0.48857,-0.756542,-0.070655
3,3,201901,0,0,False,True,False,True,False,False,...,-0.804146,-0.525393,-0.535457,-0.500012,0.0,-1.037614,-0.488194,-0.48857,-0.756542,-0.069199
4,4,201901,0,1,False,True,False,True,False,False,...,-0.804146,-0.525393,-0.535457,0.869697,0.0,-1.037614,-0.488194,-0.48857,-0.756542,-0.070655
